# Existe alguma análise específica que você gostaria de acrescentar?

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, inspect 
import import_ipynb
from Questao_02 import *

importing Jupyter notebook from Questao_02.ipynb
['notas']


In [2]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL

In [3]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.rename(columns={'id_professor(a)': 'id_professor'}, inplace=True)
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


In [4]:
# importa tabela de estudantes
tabela_estudantes = pd.read_excel('files/tabela_estudante.xlsx', index_col=False) # Importando tabela de alunos
tabela_estudantes.set_index('Unnamed: 0', inplace=True)
tabela_estudantes.index.rename('id', inplace=True)
tabela_estudantes.to_sql('estudantes', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['estudantes', 'notas']


In [69]:
# Pega a lista de todas as turmas presentes na tabela estudantes
def pega_todos_nro_turma():
    tabela_nro_turmas = tabela_estudantes['nro_turma'].drop_duplicates()
    return tabela_nro_turmas

# Ordenando a coluna de index por ondem crescente
def tratando_coluna_index(tabela_nro_turmas):
    tabela_nro_turmas = pega_todos_nro_turma()
    tabela_nro_turmas.index = range(1, tabela_nro_turmas.shape[0] + 1)
    return tabela_nro_turmas

# Separa todos os alunos por turma
def separa_alunos_por_turma(nro_turma: int):
    query = f'select * from estudantes where nro_turma = {nro_turma}'
    alunos = pd.read_sql(query, engine)
    return alunos 

# Pega lista de alunos por turma
def pega_alunos_por_turma(nro_turma: int):
    alunos = separa_alunos_por_turma(nro_turma)
    return alunos

# Calcula a media de todos os alunos semelhante a questão 2, foi ultilizado funçoes do arquivo da questão 2
def calcula_media_todos_alunos():
    media_nota_dos_alunos = []
    qtd_alunos = tabela_alunos.shape[0] + 1
    for i in range(1, qtd_alunos):
        media_nota_dos_alunos.append(media_notas_aluno_selecionado(traz_nome_aluno(i)))
    return media_nota_dos_alunos

# Pega a media das notas de cada aluno por turma
def pega_media_dos_alunos_da_turma(nro_turma: id):
    notas_turma = []
    
    alunos = pega_alunos_por_turma(nro_turma)
    list_alunos = list(alunos.id - 1)
    
    media_de_todos = calcula_media_todos_alunos()
    tamanho_lista = len(list_alunos)
    
    for i in range(tamanho_lista):
        notas_turma.append(media_de_todos[list_alunos[i]])
    
    return notas_turma 

# Calcula a media das notas medias de cada aluno da turma
def calcula_media_de_notas_turma(nro_turma: id):
    notas_turma = pega_media_dos_alunos_da_turma(nro_turma)
    notas_turma = np.mean(notas_turma)
    return notas_turma

def calcula_frequencia_por_turma(nro_turma: id):
    media_alunos_turma = []
    alunos = list(separa_alunos_por_turma(nro_turma)['id'])
    tabela = tabela_notas[['id_estudante', 'faltas']]
    tabela = tabela[tabela.id_estudante.isin(alunos)]
    for i in alunos:
        tabela_aux = tabela[tabela['id_estudante'] == i]
        media_alunos_turma.append(tabela_aux['faltas'].mean())
        media_turma = np.mean(media_alunos_turma)
    
    return media_turma

### Qual turma teve o melhor desemepenho em notas?

In [81]:
medias_por_turma = []
Turmas = ['Estatística - 12', 'Matemática Aplicada - 20', 'Matemática Aplicada - 21', 'Estatística - 13']

list_turmas = list(pega_todos_nro_turma())
qtd_turmas = len(list_turmas)

for i in range(qtd_turmas):
    medias_por_turma.append(calcula_media_de_notas_turma(list_turmas[i]))
    
exporta_para_excel(Turmas, 'Turma', medias_por_turma, 'Média das Notas por Turma', 'files/files_questions/question_9/media_por_turma.xlsx')

### Qual turma teve menos faltas? 

In [82]:
frequencia_por_turma = []

for i in list_turmas:
    freq_turma_selecionada = calcula_frequencia_por_turma(i)
    frequencia_por_turma.append(freq_turma_selecionada)
    
exporta_para_excel(Turmas, 'Turma', frequencia_por_turma, 'Média de faltas por Turma', 'files/files_questions/question_9/frequencia_por_turma.xlsx')